In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import collections
import cv2
import datetime
import math
import numpy
import os
import pathlib
import re
import skimage.io
import skimage.transform
import sklearn.model_selection
import tensorflow
import time

from albumentations import (
    Compose, HorizontalFlip, ShiftScaleRotate, ElasticTransform,
    RandomBrightness, RandomContrast, RandomGamma
)
from IPython.display import Markdown as md
from markdownTable import markdownTable

from files import create_folder, save_fit_history, save_lossgraph, save_figs
from metrics import dice_coef, jaccard_distance
from model import evaluate, unet_model, get_loss_function
from AugmentationSequence import AugmentationSequence

# GPU

In [2]:
gpus = tensorflow.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            print(f"GPU: {gpu.name}")
            tensorflow.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

GPU: /physical_device:GPU:0


In [3]:
cfg = {
    "channel": 3,
    "batch_size": 4,
    "fold": 5,
    "epochs": 75,
    "image_size": 400,
    "learning_rate": 0.001,
    "random_state": 1234,
    "test_size": 0.2,
    "val_size": 0.05,
    "path_dataset": "dataset",
    "path_out": "out",
    "loss_function": "dice"
}
images_folder = os.path.join(cfg["path_dataset"], "original")
masks_folder = os.path.join(cfg["path_dataset"], "mask")

In [4]:
list_labels = list([])
list_images = list([])
list_images_names = list([])
for file in pathlib.Path(masks_folder).rglob("*"):
    mask = skimage.io.imread(str(file.resolve()))
    mask = numpy.float32(mask > 200)
    list_labels.append(mask)

    image = skimage.io.imread(os.path.join(images_folder, file.name))
    image = skimage.transform.resize(image, (cfg["image_size"], cfg["image_size"]), anti_aliasing=True)
    image = numpy.float32(image)
    list_images.append(image)

    list_images_names.append(str(file.stem))

print(len(list_labels), len(list_images), len(list_images_names))

375 375 375


In [5]:
x = numpy.array(list_images).reshape((len(list_images), cfg["image_size"], cfg["image_size"], cfg["channel"]))
y = numpy.array(list_labels).reshape((len(list_labels), cfg["image_size"], cfg["image_size"], 1))

print(x.shape, y.shape)

(375, 400, 400, 3) (375, 400, 400, 1)


In [6]:
kf = sklearn.model_selection.KFold(n_splits=cfg["fold"], shuffle=True, random_state=cfg["random_state"])
l = list([])
for (train_index, test_index) in kf.split(x):
    l = l + test_index.tolist()
print(len(list(set(l))))
print(collections.Counter(sorted(l)))

375
Counter({0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 1, 62: 1, 63: 1, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 1, 91: 1, 92: 1, 93: 1, 94: 1, 95: 1, 96: 1, 97: 1, 98: 1, 99: 1, 100: 1, 101: 1, 102: 1, 103: 1, 104: 1, 105: 1, 106: 1, 107: 1, 108: 1, 109: 1, 110: 1, 111: 1, 112: 1, 113: 1, 114: 1, 115: 1, 116: 1, 117: 1, 118: 1, 119: 1, 120: 1, 121: 1, 122: 1, 123: 1, 124: 1, 125: 1, 126: 1, 127: 1, 128: 1, 129: 1, 130: 1, 131: 1, 132: 1, 133: 1, 134: 1, 135: 1, 136: 1, 1

In [7]:
models = []
list_evaluate = list([])
current_datetime = datetime.datetime.now().strftime('%d-%m-%Y-%H-%M-%S')
path = os.path.join(cfg["path_out"], current_datetime)
create_folder(list([path]))
for fold, (train_index, test_index) in enumerate(kf.split(x)):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(x_train, y_train, test_size=cfg["val_size"], random_state=cfg["random_state"])

    print(x_train.shape)
    print(x_val.shape)
    print(x_test.shape)
    print(x.shape)

    path_fold = os.path.join(path, str(fold))
    create_folder(list([path_fold]))

    augment = Compose([
        HorizontalFlip(),
        ShiftScaleRotate(rotate_limit=45, border_mode=cv2.BORDER_CONSTANT),
        ElasticTransform(border_mode=cv2.BORDER_CONSTANT),
        RandomBrightness(),
        RandomContrast(),
        RandomGamma()
    ])
    steps_per_epoch = math.ceil(x_train.shape[0] / cfg["batch_size"])
    train_generator = AugmentationSequence(x_train, y_train, cfg["batch_size"], augment)
    reduce_learning_rate = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=3, verbose=1)
    filename_model = os.path.join(path_fold, "unet.h5")
    checkpointer = tensorflow.keras.callbacks.ModelCheckpoint(filename_model, verbose=1, save_best_only=True)
    strategy = tensorflow.distribute.MirroredStrategy()

    with strategy.scope():
        model = unet_model(cfg)
        adam_opt = tensorflow.keras.optimizers.Adam(learning_rate=cfg["learning_rate"])
        model.compile(optimizer=adam_opt, loss=get_loss_function(cfg["loss_function"]),
                      metrics=[dice_coef, jaccard_distance, tensorflow.keras.metrics.Precision(),
                               tensorflow.keras.metrics.Recall()])

    tensorflow.keras.backend.clear_session()
    start_time = time.time()
    fit = model.fit(train_generator,
              steps_per_epoch=steps_per_epoch,
              epochs=cfg["epochs"],
              validation_data=(x_val, y_val),
               callbacks=[checkpointer, reduce_learning_rate]
              )
    end_time = time.time() - start_time

    save_fit_history(fold, fit, path_fold)
    save_lossgraph(fold, fit, path_fold)
    list_evaluate.append(evaluate(end_time, fold, model, x_train, x_val, x_test, y_train, y_val, y_test))

    models.append(model)

    # model = tensorflow.keras.models.load_model("unet.h5", custom_objects = {"jaccard_distance_loss": jaccard_distance_loss, "dice_coef": dice_coef, "jaccard_distance": jaccard_distance })

    save_figs(list_images_names, model, path_fold, x)

(285, 400, 400, 3)
(15, 400, 400, 3)
(75, 400, 400, 3)
(375, 400, 400, 3)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/d

C:\Users\xandao\anaconda3\lib\site-packages\albumentations\augmentations\transforms.py:1613: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
C:\Users\xandao\anaconda3\lib\site-packages\albumentations\augmentations\transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
36/36 [==============================] - ETA: 0s - loss: 0.5177 - dice_coef: 0.4816 - jaccard_distance: 0.3241 - precision: 0.4401 - recall: 0.9420
Epoch 1: val_loss improved from inf to 0.98885, saving model to out\06-08-2022-10-35-05\0\unet.h5
36/36 [==============================] - 88s 2s/step - loss: 0.5177 - dice_coef: 0.4816 - jaccard_distance: 0.3241 - precision: 0.4401 - recall: 0.9420 - val_loss: 0.9889 - val_dice_coef: 0.0111 - val_jaccard_distance: 0.0056 - val_precision: 1.0000 - val_recall: 4.8321e-05 - lr: 0.0010
out\06-08-2022-10-35-05\0\fold0-fit.pckl creat

In [8]:
model_cfg = {
    "batch_size": cfg["batch_size"],
    "epochs": cfg["epochs"],
    "learning_rate": cfg["learning_rate"],
    "loss_function": cfg["loss_function"],
}

image_cfg = {
    "channel": cfg["channel"],
    "image_size": cfg["image_size"],
}

test_cfg = {
    "fold": cfg["fold"],
    "test_size": cfg["test_size"],
    "val_size": cfg["val_size"],
    "random_state": cfg["random_state"],
}

other_cfg = {
    "path_dataset": cfg["path_dataset"],
    "path_out": cfg["path_out"]
}

filename_cfg = os.path.join(path, "cfg.md")
with open(filename_cfg, "w") as file:
    file.write("### model\n\n")
    file.write(re.sub(r"```$", "\n```", markdownTable(list([model_cfg])).getMarkdown()))
    file.write("\n\n### image\n\n")
    file.write(re.sub(r"```$", "\n```", markdownTable(list([image_cfg])).getMarkdown()))
    file.write("\n\n### test\n\n")
    file.write(re.sub(r"```$", "\n```", markdownTable(list([test_cfg])).getMarkdown()))
    file.write("\n\n### other\n\n")
    file.write(re.sub(r"```$", "\n```", markdownTable(list([other_cfg])).getMarkdown()))
    file.close()

In [9]:
def get_mean(key, list_evaluate):
    return numpy.mean(list([evaluate[key] for evaluate in list_evaluate]))

def get_std(key, list_evaluate):
    return numpy.std(list([evaluate[key] for evaluate in list_evaluate]))

mean_metrics_train = {
    "loss_train": get_mean("loss_train", list_evaluate),
    "dice_train": get_mean("dice_train", list_evaluate),
    "jaccard_train": get_mean("jaccard_train", list_evaluate),
    "precision_train": get_mean("precision_train", list_evaluate),
    "recall_train": get_mean("recall_train", list_evaluate)
}

std_metrics_train = {
    "loss_train": get_std("loss_train", list_evaluate),
    "dice_train": get_std("dice_train", list_evaluate),
    "jaccard_train": get_std("jaccard_train", list_evaluate),
    "precision_train": get_std("precision_train", list_evaluate),
    "recall_train": get_std("recall_train", list_evaluate)
}

mean_metrics_val = {
    "loss_val": get_mean("loss_val", list_evaluate),
    "dice_val": get_mean("dice_val", list_evaluate),
    "jaccard_val": get_mean("jaccard_val", list_evaluate),
    "precision_val": get_mean("precision_val", list_evaluate),
    "recall_val": get_mean("recall_val", list_evaluate)
}

std_metrics_val = {
    "loss_val": get_std("loss_val", list_evaluate),
    "dice_val": get_std("dice_val", list_evaluate),
    "jaccard_val": get_std("jaccard_val", list_evaluate),
    "precision_val": get_std("precision_val", list_evaluate),
    "recall_val": get_std("recall_val", list_evaluate)
}

mean_metrics_test = {
    "loss_test": get_mean("loss_test", list_evaluate),
    "dice_test": get_mean("dice_test", list_evaluate),
    "jaccard_test": get_mean("jaccard_test", list_evaluate),
    "precision_test": get_mean("precision_test", list_evaluate),
    "recall_test": get_mean("recall_test", list_evaluate)
}

std_metrics_test = {
    "loss_test": get_std("loss_test", list_evaluate),
    "dice_test": get_std("dice_test", list_evaluate),
    "jaccard_test": get_std("jaccard_test", list_evaluate),
    "precision_test": get_std("precision_test", list_evaluate),
    "recall_test": get_std("recall_test", list_evaluate)
}

filename_mean_std = os.path.join(path, "mean_std.md")
with open(filename_mean_std, "w") as file:
    file.write("### train\n\n")
    file.write(re.sub(r"```$", "\n```\n", markdownTable(list([mean_metrics_train])).getMarkdown()))
    file.write(re.sub(r"```$", "\n```\n", markdownTable(list([std_metrics_train])).getMarkdown()))
    file.write("\n\n### val\n\n")
    file.write(re.sub(r"```$", "\n```\n", markdownTable(list([mean_metrics_val])).getMarkdown()))
    file.write(re.sub(r"```$", "\n```\n", markdownTable(list([std_metrics_val])).getMarkdown()))
    file.write("\n\n### test\n\n")
    file.write(re.sub(r"```$", "\n```\n", markdownTable(list([mean_metrics_test])).getMarkdown()))
    file.write(re.sub(r"```$", "\n```\n", markdownTable(list([std_metrics_test])).getMarkdown()))
    file.close()

md(re.sub(r"```$", "\n```", markdownTable(list([mean_metrics_train])).getMarkdown()))

```
+--------------------------------------------------------------------------------------------------+
|    loss_train    |     dice_train    |   jaccard_train   |  precision_train |    recall_train    |
+------------------+-------------------+-------------------+------------------+--------------------+
|0.9222040772438049|0.07752200216054916|0.04064466431736946|0.7976624369621277|0.025603370741009712|
+--------------------------------------------------------------------------------------------------+
```

In [10]:
md(re.sub(r"```$", "\n```", markdownTable(list([std_metrics_train])).getMarkdown()))

```
+----------------------------------------------------------------+
|loss_train|dice_train|jaccard_train|precision_train|recall_train|
+----------+----------+-------------+---------------+------------+
|    0.0   |    0.0   |     0.0     |      0.0      |     0.0    |
+----------------------------------------------------------------+
```

In [11]:
md(re.sub(r"```$", "\n```", markdownTable(list([mean_metrics_val])).getMarkdown()))

```
+-------------------------------------------------------------------------------------------------+
|     loss_val     |      dice_val      |     jaccard_val    |precision_val|      recall_val      |
+------------------+--------------------+--------------------+-------------+----------------------+
|0.9888513088226318|0.011148706078529358|0.005606691353023052|     1.0     |4.8321380745619535e-05|
+-------------------------------------------------------------------------------------------------+
```

In [53]:
md(re.sub(r"```$", "\n```", markdownTable(list([std_metrics_val])).getMarkdown()))

```
+-------------------------------------------------------------+
| loss_test| dice_test|jaccard_test|precision_test|recall_test|
+----------+----------+------------+--------------+-----------+
|0.84 (0.0)|0.27 (0.0)| 0.16 (0.0) |  0.24 (0.0)  | 0.01 (0.0)|
+-------------------------------------------------------------+```

In [12]:
md(re.sub(r"```$", "\n```", markdownTable(list([mean_metrics_test])).getMarkdown()))

```
+--------------------------------------------------------------------------------------------------+
|     loss_test    |     dice_test     |    jaccard_test   |  precision_test  |     recall_test    |
+------------------+-------------------+-------------------+------------------+--------------------+
|0.9262468218803406|0.05762256309390068|0.03073793463408947|0.9211751818656921|0.034407466650009155|
+--------------------------------------------------------------------------------------------------+
```

In [13]:
md(re.sub(r"```$", "\n```", markdownTable(list([std_metrics_test])).getMarkdown()))

```
+-----------------------------------------------------------+
|loss_test|dice_test|jaccard_test|precision_test|recall_test|
+---------+---------+------------+--------------+-----------+
|   0.0   |   0.0   |     0.0    |      0.0     |    0.0    |
+-----------------------------------------------------------+
```

In [15]:
for evaluate in list_evaluate:
    filename_fold = os.path.join(path, str(evaluate["fold"]), "metrics.md")

    info = {
        "fold": evaluate["fold"],
        "time": evaluate["time"]
    }

    metrics_train = {
        "loss_train": evaluate["loss_train"],
        "dice_train": evaluate["dice_train"],
        "jaccard_train": evaluate["jaccard_train"],
        "precision_train": evaluate["precision_train"],
        "recall_train": evaluate["recall_train"],
    }

    metrics_val = {
        "loss_val": evaluate["loss_val"],
        "dice_val": evaluate["dice_val"],
        "jaccard_val": evaluate["jaccard_val"],
        "precision_val": evaluate["precision_val"],
        "recall_val": evaluate["recall_val"],
    }

    metrics_test = {
        "loss_test": evaluate["loss_test"],
        "dice_test": evaluate["dice_test"],
        "jaccard_test": evaluate["jaccard_test"],
        "precision_test": evaluate["precision_test"],
        "recall_test": evaluate["recall_test"],
    }

    with open(filename_fold, "w") as file:
        file.write("### info\n\n")
        file.write(re.sub(r"```$", "\n```", markdownTable(list([info])).getMarkdown()))
        file.write("\n\n### train\n\n")
        file.write(re.sub(r"```$", "\n```", markdownTable(list([metrics_train])).getMarkdown()))
        file.write("\n\n### val\n\n")
        file.write(re.sub(r"```$", "\n```", markdownTable(list([metrics_val])).getMarkdown()))
        file.write("\n\n### test\n\n")
        file.write(re.sub(r"```$", "\n```", markdownTable(list([metrics_test])).getMarkdown()))
        file.close()